# GUI for Character Recognition
Tkinter-based GUI for drawing and recognizing characters.

In [1]:
import os
import io
import numpy as np
from PIL import Image, ImageOps, ImageDraw, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox
import tensorflow as tf
from tensorflow import keras

# CORRECTED Configuration - matches training exactly
MODEL_PATH = r"E:\Folder\JEC Internship\Character_Recognition_Notebooks_main\handwritten_cnn_model3.h5"
IMG_SIZE = (64, 64)  # FIXED: was (160, 160) - causing shape mismatch

# Class names for 62 classes
CLASS_NAMES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] + \
              [chr(c) for c in range(ord('A'), ord('Z')+1)] + \
              [chr(c) for c in range(ord('a'), ord('z')+1)]

# Load model
try:
    model = keras.models.load_model(MODEL_PATH)
    print(f"✅ Model loaded successfully")
    print(f"Model input shape: {model.input_shape}")
except Exception as e:
    raise SystemExit(f"❌ Failed to load model: {e}")

# GUI Configuration
CANVAS_SIZE = 420
BG_COLOR = "white"
INK_COLOR = "black"
INK_WIDTH = 18

root = tk.Tk()
root.title("Handwritten Character Recognition ")
root.resizable(False, False)

# Canvas for drawing
canvas = tk.Canvas(root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg=BG_COLOR, cursor="cross")
canvas.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

# Backing image for clean pixel capture
image1 = Image.new("RGB", (CANVAS_SIZE, CANVAS_SIZE), BG_COLOR)
draw = ImageDraw.Draw(image1)

last_x, last_y = None, None

def clear_canvas():
    global image1, draw
    canvas.delete("all")
    image1 = Image.new("RGB", (CANVAS_SIZE, CANVAS_SIZE), BG_COLOR)
    draw = ImageDraw.Draw(image1)

def on_button_press(event):
    global last_x, last_y
    last_x, last_y = event.x, event.y

def on_move(event):
    global last_x, last_y
    if last_x is not None and last_y is not None:
        canvas.create_line(last_x, last_y, event.x, event.y, 
                          fill=INK_COLOR, width=INK_WIDTH, capstyle=tk.ROUND, smooth=True)
        draw.line((last_x, last_y, event.x, event.y), fill=INK_COLOR, width=INK_WIDTH)
        last_x, last_y = event.x, event.y

def on_button_release(event):
    global last_x, last_y
    last_x, last_y = None, None

# Bind drawing events
canvas.bind("<Button-1>", on_button_press)
canvas.bind("<B1-Motion>", on_move)  
canvas.bind("<ButtonRelease-1>", on_button_release)

def crop_to_content(pil_img: Image.Image):
    """Crop to drawn content using threshold"""
    gray = pil_img.convert("L")
    arr = np.array(gray)
    mask = (arr < 250).astype(np.uint8) * 255
    
    if mask.sum() == 0:
        return pil_img
        
    ys, xs = np.where(mask > 0)
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    
    pad = max(4, int(0.08 * max(x1-x0+1, y1-y0+1)))
    x0 = max(0, x0 - pad)
    y0 = max(0, y0 - pad)  
    x1 = min(arr.shape[1]-1, x1 + pad)
    y1 = min(arr.shape[0]-1, y1 + pad)
    
    return pil_img.crop((x0, y0, x1+1, y1+1))

def preprocess_for_model(pil_img: Image.Image):
    """CORRECTED: Preprocess exactly like training - no MobileNetV2!"""
    img = pil_img.convert("RGB")
    img = crop_to_content(img)
    
    # Pad to square (white background)  
    w, h = img.size
    side = max(w, h)
    bg = Image.new("RGB", (side, side), "white")
    bg.paste(img, ((side - w)//2, (side - h)//2))
    
    # FIXED: Resize to 64x64 (not 160x160)
    img = bg.resize(IMG_SIZE, Image.LANCZOS)
    
    # FIXED: Simple array conversion (model handles rescaling)
    arr = np.array(img).astype("float32")  # Keep 0-255, no manual scaling
    arr = np.expand_dims(arr, axis=0)  # (1, 64, 64, 3)
    
    return arr, img

def predict_canvas():
    """Predict drawn character"""
    try:
        arr, preview = preprocess_for_model(image1)
        print(f"Input shape: {arr.shape}")  # Should be (1, 64, 64, 3)
        
        probs = model.predict(arr, verbose=0)[0]
        idx = int(np.argmax(probs))
        top1 = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
        conf = float(probs[idx])
        
        # Top-5 predictions
        top5_idx = np.argsort(probs)[-5:][::-1]
        top5 = [(CLASS_NAMES[i] if i < len(CLASS_NAMES) else str(i), float(probs[i])) 
                for i in top5_idx]
        
        # Show results
        msg_lines = [f"Prediction: {top1} (confidence: {conf:.2%})", "", "Top-5:"]
        for lbl, p in top5:
            msg_lines.append(f"  {lbl}: {p:.2%}")
        
        messagebox.showinfo("Prediction", "\n".join(msg_lines))
        
    except Exception as e:
        messagebox.showerror("Error", f"Prediction failed: {e}")

def load_image_file():
    """Load image from file"""
    fpath = filedialog.askopenfilename(
        title="Select an image",
        filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp")]
    )
    if not fpath:
        return
        
    img = Image.open(fpath).convert("RGB").resize((CANVAS_SIZE, CANVAS_SIZE), Image.LANCZOS)
    clear_canvas()
    
    canvas_img = ImageTk.PhotoImage(img)
    canvas.create_image(0, 0, anchor=tk.NW, image=canvas_img)
    canvas.image = canvas_img  # Keep reference
    
    # Copy to backing image for prediction
    global image1, draw
    image1 = img.copy()
    draw = ImageDraw.Draw(image1)

# Create buttons
btn_predict = tk.Button(root, text="Recognize", command=predict_canvas, width=16)
btn_clear = tk.Button(root, text="Clear", command=clear_canvas, width=16) 


btn_predict.grid(row=1, column=0, padx=10, pady=(0,10), sticky="ew")
btn_clear.grid(row=1, column=1, padx=10, pady=(0,10), sticky="ew")


print("✅ GUI ready - drawing canvas uses 64x64 preprocessing")
root.mainloop()


✅ Model loaded successfully
Model input shape: (None, 64, 64, 3)
✅ GUI ready - drawing canvas uses 64x64 preprocessing
Input shape: (1, 64, 64, 3)
Input shape: (1, 64, 64, 3)
Input shape: (1, 64, 64, 3)
